In [ ]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [ ]:
url_inicial='https://www.elsevier.es/en-revista-journal-applied-research-technology-jart-81-sumario-vol-11-num-1-S1665642313X71299'
url_root= 'https://www.elsevier.es/en-revista-journal-applied-research-technology-jart-81-sumario-vol-11-num-1-S1665642313X71299'
#url_root1='https://www.elsevier.es/en-revista-journal-applied-research-technology-jart-81-numeros-anteriores'
r = requests.get(url_inicial)

In [ ]:
soup = BeautifulSoup(r.text, 'html.parser')

In [ ]:
volumen = soup.find('div', class_='items')
vol=volumen.findAll('div',class_='item')
vo = [x.find('a').get('href')for x in vol]
vo=[urljoin(url_root,i) for i in vo]
vo

In [ ]:
def get_url_items(sopa, url): 
    volumen = soup.find('div', class_='items')
    vol=volumen.findAll('div',class_='item')
    vo = [x.find('a').get('href')for x in vol]
    vo=[urljoin(url_root,i) for i in vo]
    return vo

In [ ]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

In [ ]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

In [ ]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [ ]:
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tr='Journal of Applied Research and Technology'
    a='I. Físico Matemáticas y Ciencias de la Tierra'
    te='Ciencia de Los Materiales, Ciencias de la Computación, Ingeniería Industrial, Ingeniería Electronica, Ciencias E Ingeniería en Instrumentación'
    ul='http://www.elsevier.es/en-revista-journal-of-applied-research-and-81#'
    s_item=BeautifulSoup(r.text,'html.parser')
    tituloR=tr
    if tituloR:
        content_book['Titulo Revista']=tituloR
    else :
        content_book['Titulo Revista']=None
    area=a
    if area:
        content_book['Area']=area
    else :
        content_book['Area']=None
    tematica=te
    if tematica:
        content_book['Tematica']=tematica
    else :
        content_book['Tematica']=None
    #titulo del libro
    try:
        titulo=s_item.find('div', class_='elsevierItemTitulo').get_text(strip=True)
        content_book['Titulo Articulo']=titulo
    except AttributeError:
        content_book['Titulo Articulo']=None
    try:
        des=s_item.find('div', class_='panel-body').get_text(strip=True)
        content_book['Resumen']=des
    except AttributeError:
        content_book['Resumen']=None
    try:
        de=s_item.find('div', class_='elsevierItemResumenPalabrasResumenTexto').find('p').get_text(strip=True)
        content_book['Abstract']=de
    except AttributeError:
        content_book['Abstract']=None
    urlI=ul
    if urlI:
        content_book['Url Inicial']= urlI
    else :
        content_book['Url Inicial']=None
  
        #obtener link PDF
    try:
        ancla_link=s_item.find('div', class_='opcion imprimir')
        content_book['Link Articulo']=ancla_link.find_next_sibling('a').get('href')
    except AttributeError:
        content_book['Link Articulo']=None
    return content_book

In [ ]:
list_scraper=list_scraper[0:15]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

In [ ]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

In [ ]:
df_catalogo.to_csv('Revist4.31.csv', index=False)